# Adaptive Trial EVSI Validation Notebook

This notebook validates the Adaptive Trial EVSI implementation in voiage by demonstrating its functionality with practical examples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from voiage.methods.adaptive import adaptive_evsi
from voiage.schema import ParameterSet, TrialDesign, DecisionOption

## Define an Adaptive Trial Simulator

First, let's create a function that simulates an adaptive trial and evaluates outcomes.

In [ ]:
def simulate_adaptive_trial(psa_samples, base_design, adaptive_rules):
    """Simulate an adaptive trial and evaluate economic outcomes.
    
    This is a simplified example. In practice, this would involve:
    - Simulating patient recruitment and data accrual over time
    - Applying interim analysis rules
    - Making decisions (e.g., stop/continue, change sample size)
    - Evaluating economic model based on final trial outcome
    """
    # Extract parameters
    if isinstance(psa_samples, ParameterSet):
        params = psa_samples.parameters
    else:
        params = psa_samples
    
    # Get treatment effectiveness
    if 'effectiveness' in params:
        effectiveness = params['effectiveness']
    else:
        effectiveness = np.full_like(next(iter(params.values())), 0.7)
        
    # Get treatment costs
    if 'cost' in params:
        cost = params['cost']
    else:
        cost = np.full_like(next(iter(params.values())), 5000)
    
    # Willingness-to-pay threshold (£ per QALY)
    wtp_threshold = 20000
    
    # Calculate net benefits for each strategy
    n_samples = len(effectiveness)
    n_strategies = 2
    
    net_benefits = np.zeros((n_samples, n_strategies))
    
    # Standard care net benefit
    net_benefits[:, 0] = effectiveness * 0.8 * wtp_threshold - cost * 0.8
    # New treatment net benefit (slightly better)
    net_benefits[:, 1] = effectiveness * 1.0 * wtp_threshold - cost * 1.0
    
    # Create ValueArray
    import xarray as xr
    dataset = xr.Dataset(
        {"net_benefit": (("n_samples", "n_strategies"), net_benefits)},
        coords={
            "n_samples": np.arange(n_samples),
            "n_strategies": np.arange(n_strategies),
            "strategy": ("n_strategies", ["Standard Care", "New Treatment"])
        }
    )
    
    from voiage.schema import ValueArray
    return ValueArray(dataset=dataset)

## Generate Parameter Samples

Let's generate parameter samples for our probabilistic sensitivity analysis (PSA).

In [ ]:
# Generate parameter samples
np.random.seed(42)  # For reproducibility
n_samples = 1000

# Treatment effectiveness (QALYs)
effectiveness = np.random.normal(0.7, 0.1, n_samples)

# Treatment costs (£)
cost = np.random.normal(5000, 500, n_samples)

# Create ParameterSet
parameters = {
    'effectiveness': effectiveness,
    'cost': cost
}

parameter_set = ParameterSet.from_numpy_or_dict(parameters)
print(f"Generated {parameter_set.n_samples} parameter samples")

## Define Base Trial Design

Let's define the base trial design before any adaptations.

In [ ]:
# Define base trial design
trial_arms = [
    DecisionOption(name="Standard Care", sample_size=200),
    DecisionOption(name="New Treatment", sample_size=200)
]
base_design = TrialDesign(arms=trial_arms)

print(f"Base trial design: {base_design.arms}")
print(f"Total sample size: {base_design.total_sample_size}")

## Define Adaptive Rules

Let's define the adaptive rules for our trial.

In [ ]:
# Define adaptive rules
adaptive_rules = {
    "interim_analysis_points": [0.5],  # Analyze at 50% of patients
    "early_stopping_rules": {
        "efficacy": 0.95,   # Stop for efficacy if P(new > standard) > 0.95
        "futility": 0.1     # Stop for futility if P(new > standard) < 0.1
    },
    "sample_size_adjustment": {
        "increase_if_promising": 1.5,  # Increase by 50% if promising
        "decrease_if_unpromising": 0.5  # Decrease by 50% if unpromising
    }
}

print("Adaptive rules:")
for key, value in adaptive_rules.items():
    print(f"  {key}: {value}")

## Calculate Adaptive EVSI

Now let's calculate the Expected Value of Sample Information for our adaptive trial design.

In [ ]:
# Calculate adaptive EVSI
adaptive_evsi_value = adaptive_evsi(
    adaptive_trial_simulator=simulate_adaptive_trial,
    psa_prior=parameter_set,
    base_trial_design=base_design,
    adaptive_rules=adaptive_rules,
    n_outer_loops=10,
    n_inner_loops=20
)

print(f"Adaptive EVSI: £{adaptive_evsi_value:,.0f}")

## Population-Adjusted Adaptive EVSI

Let's calculate the population-adjusted Adaptive EVSI for a larger population.

In [ ]:
# Calculate population-adjusted adaptive EVSI
adaptive_evsi_pop = adaptive_evsi(
    adaptive_trial_simulator=simulate_adaptive_trial,
    psa_prior=parameter_set,
    base_trial_design=base_design,
    adaptive_rules=adaptive_rules,
    population=100000,  # 100,000 patients
    time_horizon=10,    # 10 years
    discount_rate=0.03, # 3% discount rate
    n_outer_loops=10,
    n_inner_loops=20
)

print(f"Population-adjusted Adaptive EVSI: £{adaptive_evsi_pop:,.0f}")

## Sensitivity Analysis

Let's perform a sensitivity analysis by varying the sample size of the proposed trial.

In [ ]:
# Vary sample size and calculate adaptive EVSI
sample_sizes = [100, 200, 300, 500]
evsi_values = []

for sample_size in sample_sizes:
    trial_arms = [
        DecisionOption(name="Standard Care", sample_size=sample_size),
        DecisionOption(name="New Treatment", sample_size=sample_size)
    ]
    trial_design = TrialDesign(arms=trial_arms)
    
    evsi_val = adaptive_evsi(
        adaptive_trial_simulator=simulate_adaptive_trial,
        psa_prior=parameter_set,
        base_trial_design=trial_design,
        adaptive_rules=adaptive_rules,
        n_outer_loops=5,
        n_inner_loops=10
    )
    
    evsi_values.append(evsi_val)
    print(f"Sample size: {sample_size*2} (total), Adaptive EVSI: £{evsi_val:,.0f}")

## Visualization

Let's visualize how Adaptive EVSI changes with sample size.

In [ ]:
# Plot Adaptive EVSI vs Sample Size
plt.figure(figsize=(10, 6))
plt.plot([size*2 for size in sample_sizes], evsi_values, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Total Sample Size')
plt.ylabel('Adaptive EVSI (£)')
plt.title('Adaptive EVSI vs Sample Size for Clinical Trial')
plt.grid(True, alpha=0.3)
plt.show()

## Comparison with Non-Adaptive Design

Let's compare the adaptive design with a traditional fixed-sample design.

In [ ]:
# For comparison, let's calculate EVSI for a fixed design
from voiage.methods.sample_information import evsi

def simple_economic_model(parameter_samples):
    """Simple economic model for fixed design comparison."""
    # Extract parameters
    if isinstance(parameter_samples, ParameterSet):
        params = parameter_samples.parameters
    else:
        params = parameter_samples
    
    # Get treatment effectiveness
    if 'effectiveness' in params:
        effectiveness = params['effectiveness']
    else:
        effectiveness = np.full_like(next(iter(params.values())), 0.7)
        
    # Get treatment costs
    if 'cost' in params:
        cost = params['cost']
    else:
        cost = np.full_like(next(iter(params.values())), 5000)
    
    # Willingness-to-pay threshold (£ per QALY)
    wtp_threshold = 20000
    
    # Calculate net benefits for each strategy
    n_samples = len(effectiveness)
    n_strategies = 2
    
    net_benefits = np.zeros((n_samples, n_strategies))
    
    # Standard care net benefit
    net_benefits[:, 0] = effectiveness * 0.8 * wtp_threshold - cost * 0.8
    # New treatment net benefit (slightly better)
    net_benefits[:, 1] = effectiveness * 1.0 * wtp_threshold - cost * 1.0
    
    # Create ValueArray
    import xarray as xr
    dataset = xr.Dataset(
        {"net_benefit": (("n_samples", "n_strategies"), net_benefits)},
        coords={
            "n_samples": np.arange(n_samples),
            "n_strategies": np.arange(n_strategies),
            "strategy": ("n_strategies", ["Standard Care", "New Treatment"])
        }
    )
    
    from voiage.schema import ValueArray
    return ValueArray(dataset=dataset)

# Calculate EVSI for fixed design
fixed_evsi_value = evsi(
    economic_model=simple_economic_model,
    parameter_uncertainty=parameter_set,
    n_outer_loops=10,
    n_inner_loops=20
)

print(f"Fixed Design EVSI: £{fixed_evsi_value:,.0f}")
print(f"Adaptive Design EVSI: £{adaptive_evsi_value:,.0f}")
print(f"Difference: £{adaptive_evsi_value - fixed_evsi_value:,.0f}")

if adaptive_evsi_value > fixed_evsi_value:
    print("Adaptive design provides higher value than fixed design.")
elif adaptive_evsi_value < fixed_evsi_value:
    print("Fixed design provides higher value than adaptive design.")
else:
    print("Both designs provide equivalent value.")

## Summary

This notebook has demonstrated:

1. How to set up an adaptive trial EVSI analysis with voiage
2. How to define adaptive trial simulators
3. How to calculate adaptive EVSI for proposed trial designs
4. How to perform population-adjusted calculations
5. How to conduct sensitivity analysis on sample size
6. How to compare adaptive designs with fixed designs

The voiage adaptive trial EVSI implementation provides a flexible framework for evaluating the value of adaptive trial designs in health economics and other domains.